# Plan your trip with Kayak

The marketing team needs help on a new project. After doing some user research, the team discovered that 70% of their users who are planning a trip would like to have more information about the destination they are going to.

In addition, user research shows that people tend to be defiant about the information they are reading if they don't know the brand which produced the content.

Therefore, Kayak Marketing Team would like to create an application that will recommend where people should plan their next holidays. The application should be based on real data about:

    Weather
    Hotels in the area

The application should then be able to recommend the best destinations and hotels based on the above variables at any given time.


## Part 1 : Create a dataframe with cities

In [1]:
# Import useful libraries

import pandas as pd
import numpy as np
import requests
pd.options.display.max_columns = None

!pip install plotly -q
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = "iframe_connected"

!pip install Scrapy -q
import os 
import logging
import scrapy
from scrapy.crawler import CrawlerProcess

Marketing team wants to focus first on the best cities to travel to in France. According One Week In.com here are the top-35 cities to visit in France:

In [2]:
cities = ["Mont Saint Michel","St Malo","Bayeux","Le Havre","Rouen","Paris","Amiens","Lille","Strasbourg","Château du Haut Koenigsbourg","Colmar",
"Eguisheim","Besancon","Dijon","Annecy","Grenoble","Lyon","Gorges du Verdon","Bormes les Mimosas","Cassis","Marseille","Aix en Provence","Avignon",
"Uzès","Nîmes","Aigues Mortes","Saintes Maries de la mer","Collioure","Carcassonne","Ariège","Toulouse","Montauban","Biarritz","Bayonne",
"La Rochelle"]

In [3]:
# Dataframe creation

print("Creating dataset...")
df = pd.DataFrame(cities)
df = df.rename(columns={0:"cities"})
print("...Done.")
print()
df.head()

Creating dataset...
...Done.



,cities
0,Mont Saint Michel
1,St Malo
2,Bayeux
3,Le Havre
4,Rouen


In [4]:
# Creation a column ID

print("Creating column...")
df.insert(0,"ID",range(1, len(df) + 1))
print("...Done.")
print()
df.head()

Creating column...
...Done.



,ID,cities
0,1,Mont Saint Michel
1,2,St Malo
2,3,Bayeux
3,4,Le Havre
4,5,Rouen


## Part 2 : Get weather data with an API

In [5]:
# Replacement of spaces by + for the proper execution of the loop

print("Replacing spaces...")
df["cities"] = df["cities"].str.replace(" ","+")
print("...Done.")

#Launch of the loop on API to retrieve the geolocation of each cities
#Insertion of values in the dataframe

print("Add geolocation...")
for i in df.index:
    city = df.loc[i,"cities"]
    geolocation = requests.get('https://nominatim.openstreetmap.org/search?q={}&format=json'.format(city)).json()
    df.loc[i,"city_lat"] = geolocation[0]["lat"]
    df.loc[i,"city_lon"] = geolocation[0]["lon"]

print("...Done.")
print()
df.head()

Replacing spaces...
...Done.
Add geolocation...
...Done.



,ID,cities,city_lat,city_lon
0,1,Mont+Saint+Michel,48.6359541,-1.511459954959514
1,2,St+Malo,48.649518,-2.0260409
2,3,Bayeux,49.2764624,-0.7024738
3,4,Le+Havre,49.4938975,0.1079732
4,5,Rouen,49.4404591,1.0939658


In [6]:
# Deletion of the +

print("Deleting + ...")
df['cities'] = df['cities'].str.replace('+',' ')   
print("...Done.")
print()
df.head()

Deleting + ...
...Done.



,ID,cities,city_lat,city_lon
0,1,Mont Saint Michel,48.6359541,-1.511459954959514
1,2,St Malo,48.649518,-2.0260409
2,3,Bayeux,49.2764624,-0.7024738
3,4,Le Havre,49.4938975,0.1079732
4,5,Rouen,49.4404591,1.0939658


Determine the list of cities where the weather will be the nicest within the next 7 days

In [7]:
# Launch of the loop on API to retrieve daily weather data for each cities
#Insert values in DataFrame

print("Add daily weather data...")
for i in df.index:
    lat = df.loc[i,"city_lat"]
    long = df.loc[i,"city_lon"]
    data = requests.get("https://api.openweathermap.org/data/2.5/onecall?lat={}&lon={}&units=metric&exclude=current,minutely,hourly&lang=fr&appid=42def4a4519b0228cc6beaf04b144ed4".format(lat,long))
    
# Chosen data : felt temperature

    df.loc[i,"felt_temp_j+1"] = data.json()["daily"][1]["feels_like"]["day"]
    df.loc[i,"felt_temp_j+2"] = data.json()["daily"][2]["feels_like"]["day"]
    df.loc[i,"felt_temp_j+3"] = data.json()["daily"][3]["feels_like"]["day"]
    df.loc[i,"felt_temp_j+4"] = data.json()["daily"][4]["feels_like"]["day"]
    df.loc[i,"felt_temp_j+5"] = data.json()["daily"][5]["feels_like"]["day"]
    df.loc[i,"felt_temp_j+6"] = data.json()["daily"][6]["feels_like"]["day"]
    df.loc[i,"felt_temp_j+7"] = data.json()["daily"][7]["feels_like"]["day"]

print("...Done.")
print()
df.head()

Add daily weather data...
...Done.



,ID,cities,city_lat,city_lon,felt_temp_j+1,felt_temp_j+2,felt_temp_j+3,felt_temp_j+4,felt_temp_j+5,felt_temp_j+6,felt_temp_j+7
0,1,Mont Saint Michel,48.6359541,-1.511459954959514,10.88,4.89,3.78,1.86,2.71,3.19,5.26
1,2,St Malo,48.649518,-2.0260409,10.77,5.56,3.88,2.44,3.41,3.99,5.38
2,3,Bayeux,49.2764624,-0.7024738,7.36,5.82,2.22,1.46,2.11,1.56,3.87
3,4,Le Havre,49.4938975,0.1079732,6.79,5.50,2.40,2.29,3.10,1.46,3.56
4,5,Rouen,49.4404591,1.0939658,9.99,4.05,1.52,2.50,1.87,1.00,3.37


In [8]:
# Calculation of the average felt temperature over 7 days

print("Add average of felt temperature...")
for i in df.index:
    df.loc[i,"felt_temp_avg"] = df.loc[i,["felt_temp_j+1","felt_temp_j+2","felt_temp_j+3","felt_temp_j+4","felt_temp_j+5","felt_temp_j+6","felt_temp_j+7"]].mean()
print("...Done.")
print()
df.head()

Add average of felt temperature...
...Done.



,ID,cities,city_lat,city_lon,felt_temp_j+1,felt_temp_j+2,felt_temp_j+3,felt_temp_j+4,felt_temp_j+5,felt_temp_j+6,felt_temp_j+7,felt_temp_avg
0,1,Mont Saint Michel,48.6359541,-1.511459954959514,10.88,4.89,3.78,1.86,2.71,3.19,5.26,4.652857
1,2,St Malo,48.649518,-2.0260409,10.77,5.56,3.88,2.44,3.41,3.99,5.38,5.061429
2,3,Bayeux,49.2764624,-0.7024738,7.36,5.82,2.22,1.46,2.11,1.56,3.87,3.485714
3,4,Le Havre,49.4938975,0.1079732,6.79,5.50,2.40,2.29,3.10,1.46,3.56,3.585714
4,5,Rouen,49.4404591,1.0939658,9.99,4.05,1.52,2.50,1.87,1.00,3.37,3.471429


In [9]:
# Conversation des colonnes latitude et longitude en float pour la visualisation

print("Converting ...")
df["city_lat"] = df["city_lat"].astype(float)
df["city_lon"] = df["city_lon"].astype(float)
print("...Done.")

Converting ...
...Done.


In [10]:
# Visualization of the 35 french cities according to the weather

fig1 = px.scatter_mapbox(df, lat="city_lat", lon="city_lon", color="felt_temp_avg",hover_name="cities",
                        mapbox_style="carto-positron", zoom=4)
fig1.show()

In [11]:
# Determining the top 5 french cities according to the weather

Top5cities = df.sort_values(["felt_temp_avg"], ascending=False)
Top5cities.head()

,ID,cities,city_lat,city_lon,felt_temp_j+1,felt_temp_j+2,felt_temp_j+3,felt_temp_j+4,felt_temp_j+5,felt_temp_j+6,felt_temp_j+7,felt_temp_avg
18,19,Bormes les Mimosas,43.150697,6.341928,13.82,13.53,11.41,8.51,8.72,9.09,5.30,10.054286
33,34,Bayonne,43.493338,-1.475099,14.83,12.32,5.11,7.48,9.95,7.05,10.43,9.595714
32,33,Biarritz,43.483252,-1.559278,14.54,12.18,5.06,7.31,10.10,6.99,10.47,9.521429
27,28,Collioure,42.525050,3.083155,15.28,12.79,9.72,3.87,8.68,4.99,10.24,9.367143
19,20,Cassis,43.218178,5.553394,12.88,14.19,10.65,7.28,6.88,7.16,4.08,9.017143


In [12]:
#  Visualization of the top 5 french cities according to the weather
fig2 = px.scatter_mapbox(Top5cities[0:5], lat="city_lat", lon="city_lon", color="felt_temp_avg",hover_name="cities", size="felt_temp_avg",
                        mapbox_style="carto-positron", zoom=5)
fig2.show()

In [13]:
# Save all the results in a .csv file

df.to_csv("weather_data.csv")

## Part 3 : Scrape Booking.com

In [14]:
class Booking(scrapy.Spider):

    name = "booking"

    start_urls = [
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3D9a93888d707e0185%26%3B&ss=Le+Mont-Saint-Michel%2C+Basse-Normandie%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=mont+s&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=900039327&dest_type=city&place_id_lat=48.636&place_id_lon=-1.511&search_pageview_id=483d88b5c858000d&search_selected=true&search_pageview_id=483d88b5c858000d&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3D23a888bbff44010b%26%3B&ss=Saint-Malo%2C+Bretagne%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=st&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1466824&dest_type=city&place_id_lat=48.649616&place_id_lon=-2.025677&search_pageview_id=49c18913f3bc0253&search_selected=true&search_pageview_id=49c18913f3bc0253&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D1000&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3Daf4089ac072401ed%26%3B&ss=Bayeux%2C+Basse-Normandie%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=baye&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1410836&dest_type=city&place_id_lat=49.27605&place_id_lon=-0.70229&search_pageview_id=03ce89e512f9004f&search_selected=true&search_pageview_id=03ce89e512f9004f&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3D721989e9b03a0145%26%3B&ss=Le+Havre%2C+Haute-Normandie%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=le+havre&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1441598&dest_type=city&place_id_lat=49.49489&place_id_lon=0.107723&search_pageview_id=59cb8a57e2ea007b&search_selected=true&search_pageview_id=59cb8a57e2ea007b&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D1000%3Bdistance%3D3000&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3D055b8a5b520d0004%26%3B&ss=Rouen%2C+Haute-Normandie%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=roue&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1462807&dest_type=city&place_id_lat=49.44046&place_id_lon=1.09397&search_pageview_id=f3ab8a72a06b00c5&search_selected=true&search_pageview_id=f3ab8a72a06b00c5&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D1000&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3D4ebd8a757e40011e%26%3B&ss=Paris%2C+%C3%8Ele-de-France%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=pati&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1456928&dest_type=city&iata=PAR&place_id_lat=48.85668&place_id_lon=2.351476&search_pageview_id=6fe38abb1c2800f2&search_selected=true&search_pageview_id=6fe38abb1c2800f2&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D1000&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3D0b598ac184db01e2%26%3B&ss=Amiens%2C+Picardie%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=amiens&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1407447&dest_type=city&place_id_lat=49.895454&place_id_lon=2.299816&search_pageview_id=c3cd8aed3d81002f&search_selected=true&search_pageview_id=c3cd8aed3d81002f&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D1000&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3Df9028af16e170021%26%3B&ss=Lille%2C+Nord-Pas-de-Calais%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=lille&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1447079&dest_type=city&iata=LIL&place_id_lat=50.636845&place_id_lon=3.063454&search_pageview_id=208c8b21fceb02e0&search_selected=true&search_pageview_id=208c8b21fceb02e0&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D1000&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3De77c8b26b1d70184%26%3B&ss=Strasbourg%2C+Alsace%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=strasbou&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1471697&dest_type=city&iata=SXB&place_id_lat=48.5834&place_id_lon=7.7458&search_pageview_id=b2758b425c6d017a&search_selected=true&search_pageview_id=b2758b425c6d017a&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D1000&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3D78538b45cf880035%26%3B&ss=Ch%C3%A2teau+du+Haut-K%C5%93nigsbourg%2C+Saint-Hippolyte%2C+Alsace%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&map=1&b_h4u_keep_filters=&from_sf=1&ss_raw=chateau+du+haut&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=204055&dest_type=landmark&place_id_lat=48.24931&place_id_lon=7.34445&search_pageview_id=12018b5e82d501a3&search_selected=true&nflt=ht_id%3D204%3Bdistance%3D3000%3Bdistance%3D5000&order=score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3Dc2368b8e2ebc009e%26%3B&ss=Colmar%2C+Alsace%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=colmar&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1421049&dest_type=city&place_id_lat=48.079155&place_id_lon=7.357718&search_pageview_id=edf78bb83abb0120&search_selected=true&search_pageview_id=edf78bb83abb0120&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D1000&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3D7a6d8bd85a450067%26%3B&ss=Eguisheim%2C+Alsace%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=egui&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1425030&dest_type=city&place_id_lat=48.04231&place_id_lon=7.30543&search_pageview_id=d5228beb0fd501b0&search_selected=true&search_pageview_id=d5228beb0fd501b0&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=distance%3D1000%3Bht_id%3D204%3Bdistance%3D3000%3Bdistance%3D5000&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3Dd9008bf076cf00f3%26%3B&ss=Besan%C3%A7on%2C+Franche-Comt%C3%A9%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=besan&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1412198&dest_type=city&place_id_lat=47.240215&place_id_lon=6.023773&search_pageview_id=9d4c8c15ac3501e9&search_selected=true&search_pageview_id=9d4c8c15ac3501e9&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D3000%3Bdistance%3D1000%3Bdistance%3D5000&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3D2f548c1a73b300e0%26%3B&ss=Dijon%2C+Bourgogne%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=dijon&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1423981&dest_type=city&place_id_lat=47.321762&place_id_lon=5.039809&search_pageview_id=7ed98c3aa18c06a6&search_selected=true&search_pageview_id=7ed98c3aa18c06a6&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D1000&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3De7ba8c3f7284002d%26%3B&ss=Annecy%2C+Rh%C3%B4ne-Alpes%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=anne&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1407760&dest_type=city&iata=NCY&place_id_lat=45.899727&place_id_lon=6.127196&search_pageview_id=766a8c565e140116&search_selected=true&search_pageview_id=766a8c565e140116&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D1000&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3D82dc8c76f1510158%26%3B&ss=Grenoble%2C+Rh%C3%B4ne-Alpes%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=greno&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1430647&dest_type=city&place_id_lat=45.192505&place_id_lon=5.724973&search_pageview_id=ae198c78cd7c0047&search_selected=true&search_pageview_id=ae198c78cd7c0047&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D1000%3Bdistance%3D3000&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3D4d538c7c234203d6%26%3B&ss=Lyon%2C+Rh%C3%B4ne-Alpes%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=lyon&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1448468&dest_type=city&iata=LYS&place_id_lat=45.757767&place_id_lon=4.832204&search_pageview_id=e6518ca46763000d&search_selected=true&search_pageview_id=e6518ca46763000d&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D1000&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3Dc6358ca7ca6e0215%26%3B&ss=Gorges+du+Verdon%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=gorge+du+ver&ac_position=1&ac_langcode=fr&ac_click_type=b&dest_id=2746&dest_type=region&place_id_lat=43.78561&place_id_lon=6.126267&search_pageview_id=1b4a8d07341e0144&search_selected=true&search_pageview_id=1b4a8d07341e0144&ac_suggestion_list_length=2&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204&order=score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3De1028d3f2a7c00be%26%3B&ss=Bormes-les-Mimosas%2C+Provence-Alpes-C%C3%B4te+d%27Azur%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=borme&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1413801&dest_type=city&place_id_lat=43.1507&place_id_lon=6.34193&search_pageview_id=df1f8d4934aa00ca&search_selected=true&search_pageview_id=df1f8d4934aa00ca&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D1000%3Bdistance%3D3000%3Bdistance%3D5000&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&src=index&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3D32eb8d4e446d0308%26%3B&ss=Cassis%2C+Provence-Alpes-C%C3%B4te+d%27Azur%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=cassis&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1416912&dest_type=city&place_id_lat=43.21513&place_id_lon=5.53712&search_pageview_id=ddf78d85c9e300b4&search_selected=true&search_pageview_id=ddf78d85c9e300b4&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D3000%3Bdistance%3D1000%3Bdistance%3D5000",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3D1d528d8a2265016b%26%3B&ss=Marseille%2C+Provence-Alpes-C%C3%B4te+d%27Azur%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=marsei&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1449947&dest_type=city&iata=MRS&place_id_lat=43.2983&place_id_lon=5.37798&search_pageview_id=0dc78da45f600011&search_selected=true&search_pageview_id=0dc78da45f600011&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D1000&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3D61bd8da7774201e3%26%3B&ss=Aix-en-Provence%2C+Provence-Alpes-C%C3%B4te+d%27Azur%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=aix&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1406939&dest_type=city&place_id_lat=43.526314&place_id_lon=5.445464&search_pageview_id=48248dc3a602056a&search_selected=true&search_pageview_id=48248dc3a602056a&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D1000&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3D88ec8dc7dde3019b%26%3B&ss=Avignon%2C+Provence-Alpes-C%C3%B4te+d%27Azur%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=avign&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1409631&dest_type=city&iata=AVN&place_id_lat=43.949&place_id_lon=4.807508&search_pageview_id=a6c18ddc291501ac&search_selected=true&search_pageview_id=a6c18ddc291501ac&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D1000&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3D9d918de180ad0399%26%3B&ss=Uz%C3%A8s%2C+Languedoc-Roussillon%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=uze&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1474231&dest_type=city&place_id_lat=44.01217&place_id_lon=4.41967&search_pageview_id=96be8df794cc0228&search_selected=true&search_pageview_id=96be8df794cc0228&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D1000%3Bdistance%3D3000%3Bdistance%3D5000&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3D14898dfbb95600d0%26%3B&ss=N%C3%AEmes%2C+Languedoc-Roussillon%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=nime&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1455068&dest_type=city&iata=FNI&place_id_lat=43.83742&place_id_lon=4.36007&search_pageview_id=c7fb8e218466019f&search_selected=true&search_pageview_id=c7fb8e218466019f&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D1000&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3Defba8e245996002f%26%3B&ss=Aigues-Mortes%2C+Languedoc-Roussillon%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=aigue&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1406800&dest_type=city&place_id_lat=43.566887&place_id_lon=4.190193&search_pageview_id=df138e3b8bcc010d&search_selected=true&search_pageview_id=df138e3b8bcc010d&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D1000%3Bdistance%3D3000%3Bdistance%3D5000&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3D13ca8e41a87600f9%26%3B&ss=Les+Saintes-Maries-de-la-Mer%2C+Provence-Alpes-C%C3%B4te+d%27Azur%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=sainte&ac_position=2&ac_langcode=fr&ac_click_type=b&dest_id=-1465138&dest_type=city&place_id_lat=43.45233&place_id_lon=4.42871&search_pageview_id=03968e5cdf670103&search_selected=true&search_pageview_id=03968e5cdf670103&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D1000%3Bdistance%3D3000&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3D23058e66dc8f0012%26%3B&ss=Collioure%2C+Languedoc-Roussillon%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=coll&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1421032&dest_type=city&place_id_lat=42.52453&place_id_lon=3.082255&search_pageview_id=e0a88e837bcd00de&search_selected=true&search_pageview_id=e0a88e837bcd00de&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D1000&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3D6eda8ea69ac3000a%26%3B&ss=Carcassonne%2C+Languedoc-Roussillon%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=carca&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1416701&dest_type=city&iata=CCF&place_id_lat=43.21304&place_id_lon=2.34911&search_pageview_id=13ca8f1fcafe0147&search_selected=true&search_pageview_id=13ca8f1fcafe0147&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D1000%3Bdistance%3D3000&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3D418f8f224f7e0099%26%3B&ss=Ari%C3%A8ge%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=ari%C3%A8ge&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=2507&dest_type=region&place_id_lat=42.89792&place_id_lon=1.612936&search_pageview_id=38328f470e310243&search_selected=true&search_pageview_id=38328f470e310243&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204&order=score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3D17e28f51e52a0299%26%3B&ss=Toulouse%2C+Midi-Pyr%C3%A9n%C3%A9es%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=toulouse&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1473166&dest_type=city&iata=TLS&place_id_lat=43.604645&place_id_lon=1.444213&search_pageview_id=e2c58f9651b80177&search_selected=true&search_pageview_id=e2c58f9651b80177&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D1000&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3D10c68f9a54fc00d4%26%3B&ss=Montauban%2C+Midi-Pyr%C3%A9n%C3%A9es%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=montau&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1452421&dest_type=city&place_id_lat=44.01758&place_id_lon=1.355&search_pageview_id=70ec8faf1af30118&search_selected=true&search_pageview_id=70ec8faf1af30118&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D1000%3Bdistance%3D3000%3Bdistance%3D5000&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3D32eb8fb49ee9028f%26%3B&ss=Biarritz%2C+Aquitaine%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=biarr&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1412526&dest_type=city&iata=BIQ&place_id_lat=43.4804&place_id_lon=-1.56066&search_pageview_id=00a38fcaa8540008&search_selected=true&search_pageview_id=00a38fcaa8540008&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D1000&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3Db48c8fce241f0177%26%3B&ss=Bayonne%2C+Aquitaine%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=bayonn&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1410844&dest_type=city&place_id_lat=43.49334&place_id_lon=-1.4751&search_pageview_id=06548fe1fe33021c&search_selected=true&search_pageview_id=06548fe1fe33021c&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D1000%3Bdistance%3D3000%3Bdistance%3D5000&order=bayesian_review_score",
        "https://www.booking.com/searchresults.fr.html?label=fr-85Sbyi2evytni3mHZEi6UgS267492169156%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9056586%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o&sid=02b206a84eecccc147910770df8a2890&aid=376366&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D376366%3Blabel%3Dfr-85Sbyi2evytni3mHZEi6UgS267492169156%253Apl%253Ata%253Ap1%253Ap22.563.000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9056586%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6vvamF_no64o%3Bsid%3D02b206a84eecccc147910770df8a2890%3Bsb_price_type%3Dtotal%3Bsrpvid%3D1f088fe590f901db%26%3B&ss=La+Rochelle%2C+Poitou-Charentes%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=la+rochell&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1438604&dest_type=city&iata=LRH&place_id_lat=46.15911&place_id_lon=-1.15204&search_pageview_id=b9968ffcc1cc0064&search_selected=true&search_pageview_id=b9968ffcc1cc0064&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204%3Bdistance%3D1000&order=bayesian_review_score"
        ]
    
    def parse(self, response):
        hotels = response.css('div._fe1927d9e._0811a1b54._a8a1be610._022ee35ec.b9c27d6646.fb3c4512b4.fc21746a73')
        for hotel in hotels:
            try :
                hotel_name = hotel.css('div.fde444d7ef._c445487e2::text').get()
                hotel_url = hotel.css('a::attr(href)').get()
                hotel_describe = hotel.css('div._4abc4c3d5::text').get()
                hotel_ranking = hotel.css('div._9c5f726ff.bd528f9ea6::text').get()
                hotel_city = hotel.css('span.af1ddfc958.eba89149fb::text').get().strip()
                
            except:
                print ("Error : ", hotel)
            else:
                yield scrapy.Request(
                    hotel_url, 
                    callback=self.hotel_page,
                    meta={ 
                        'hotel_name':hotel_name, 
                        'hotel_url':hotel_url, 
                        'hotel_describe':hotel_describe, 
                        'hotel_ranking':hotel_ranking,
                        'hotel_city':hotel_city
                    }
                    )
        
    def hotel_page(self, response):

        lat_lon = response.css('a.show_on_map_hp_link').attrib['data-atlas-latlng'].split(",")

        output ={
            'hotel_name':response.meta.get('hotel_name'),
            'hotel_url':response.meta.get('hotel_url'),
            'hotel_describe':response.meta.get('hotel_describe'),
            'hotel_ranking':response.meta.get('hotel_ranking'),
            'hotel_city':response.meta.get('hotel_city'),
            'hotel_lat':lat_lon[0],
            'hotel_lon':lat_lon[1]    
        }
        return output

In [15]:
filename = "HotelsBooking.json"

if filename in os.listdir('src/'):
        os.remove('src/' + filename)

process = CrawlerProcess(settings = {
    'USER_AGENT': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:94.0) Gecko/20100101 Firefox/94.0',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        'src/' + filename : {"format": "json"},
    },
    "AUTOTHROTTLE_ENABLED": True
})

# Start the crawling
process.crawl(Booking)
process.start()

2022-01-02 11:13:10 [scrapy.utils.log] INFO: Scrapy 2.5.1 started (bot: scrapybot)
2022-01-02 11:13:10 [scrapy.utils.log] INFO: Versions: lxml 4.7.1.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.8.6 | packaged by conda-forge | (default, Oct  7 2020, 19:08:05) - [GCC 7.5.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.1.1, Platform Linux-5.4.129+-x86_64-with-glibc2.10
2022-01-02 11:13:10 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'LOG_LEVEL': 20,
 'USER_AGENT': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:94.0) '
               'Gecko/20100101 Firefox/94.0'}
2022-01-02 11:13:10 [scrapy.extensions.telnet] INFO: Telnet Password: a62eb381c9bda2bd
2022-01-02 11:13:10 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.

In [16]:
# Read file with hotels data

print("Loading dataset...")
data = pd.read_json("src/HotelsBooking.json")
print("...Done")
print()
data.head()

2022-01-02 11:29:23 [numexpr.utils] INFO: NumExpr defaulting to 4 threads.


Loading dataset...
...Done



,hotel_name,hotel_url,hotel_describe,hotel_ranking,hotel_city,hotel_lat,hotel_lon
0,Le Lithana,https://www.booking.com/hotel/fr/le-lithana.fr...,"Situé dans le centre-ville de Pontorson, l'hôt...","7,3",Pontorson,48.554097,-1.500629
1,Les Chiens du Guet,https://www.booking.com/hotel/fr/les-chiens-du...,L'établissement Les Chiens du Guet est situé a...,"7,4","Intra muros, Saint-Malo",48.647932,-2.028651
2,Chateau La Cheneviere,https://www.booking.com/hotel/fr/chateau-la-ch...,L'établissement Chateau La Cheneviere est situ...,"8,8",Port-en-Bessin-Huppain,49.332429,-0.755652
3,Hotel Les Arcades,https://www.booking.com/hotel/fr/des-arcades-r...,L'Hôtel Les Arcades est un établissement abord...,"6,2","Centre-ville de Rouen, Rouen",49.441782,1.094688
4,Hôtel Les Gens De Mer Le Havre by Popinns,https://www.booking.com/hotel/fr/les-gens-de-m...,Les Gens de Mer est situé en plein cœur du Hav...,"6,5","Perret, Le Havre",49.489918,0.103383


In [17]:
# Basics statistics

print("Number of rows : {}".format(data.shape[0]))
print()

print("Number of columns : {}".format(data.shape[1]))
print()

print("Basics statistics: ")
data_desc = data.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
print()
display(100*data.isnull().sum()/data.shape[0])

Number of rows : 773

Number of columns : 7

Basics statistics: 


,hotel_name,hotel_url,hotel_describe,hotel_ranking,hotel_city,hotel_lat,hotel_lon
count,773,773,773,772,773,773.000000,773.000000
unique,767,773,764,41,118,NaN,NaN
top,Résidence Pierre & Vacances La Petite Venise,https://www.booking.com/hotel/fr/adagio-access...,Hébergement géré par un particulier,"8,6","Centre de Toulouse, Toulouse",NaN,NaN
freq,2,1,9,66,25,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,45.946211,3.278776
std,NaN,NaN,NaN,NaN,NaN,2.554189,2.973709
min,NaN,NaN,NaN,NaN,NaN,42.523716,-2.028651
25%,NaN,NaN,NaN,NaN,NaN,43.493491,1.096934
50%,NaN,NaN,NaN,NaN,NaN,45.756037,4.362811
75%,NaN,NaN,NaN,NaN,NaN,48.583614,5.716697



Percentage of missing values: 



hotel_name        0.000000
hotel_url         0.000000
hotel_describe    0.000000
hotel_ranking     0.129366
hotel_city        0.000000
hotel_lat         0.000000
hotel_lon         0.000000
dtype: float64

In [18]:
data['hotel_city'].unique()

array(['Pontorson', 'Intra muros, Saint-Malo', 'Port-en-Bessin-Huppain',
       'Centre-ville de Rouen, Rouen', 'Perret, Le Havre',
       '5e arr., Paris', 'Amiens', 'Centre de Lille, Lille', 'Strasbourg',
       'La Vancelle', 'Colmar', 'Wintzenheim',
       'Centre-ville de Dijon, Dijon', "Centre-ville d'Annecy, Annecy",
       '1er arr., Lyon', 'Grenoble', 'Grenoble City Centre, Grenoble',
       '2e arr., Lyon', '5e arr., Lyon', 'Annecy', 'Dijon', 'Besançon',
       'Besancon Old Town, Besançon', 'Gueberschwihr', 'Wettolsheim',
       'Husseren-les-Châteaux', 'Turckheim', 'Eguisheim',
       'Centre-ville de Colmar, Colmar', 'Bergheim', 'Saint-Hippolyte',
       'Kintzheim', 'Châtenois', 'Thannenkirch', 'Rodern', 'Orschwiller',
       'Gare de Strasbourg, Strasbourg',
       'Centre de Strasbourg - Petite France - Cathédrale, Strasbourg',
       'Bourse-Esplanade, Strasbourg',
       'Quartier Européen - Orangerie, Strasbourg', 'Vieux-Lille, Lille',
       '3e arr., Paris', '4e ar

In [19]:
# Cleaning dataset

data["hotel_ranking"] = data["hotel_ranking"].str.replace(",",".")

#Mont Saint Michel

data["hotel_city"] = data["hotel_city"].str.replace("Beauvoir","Mont Saint Michel")
data["hotel_city"] = data["hotel_city"].str.replace('Le Mont-Saint-Michel',"Mont Saint Michel")
data["hotel_city"] = data["hotel_city"].str.replace('Beauvoir','Mont Saint Michel')
data["hotel_city"] = data["hotel_city"].str.replace('Pontorson','Mont Saint Michel')
data["hotel_city"] = data["hotel_city"].str.replace('Courtils','Mont Saint Michel')
data["hotel_city"] = data["hotel_city"].str.replace('Roz-sur-Couesnon','Mont Saint Michel')
data["hotel_city"] = data["hotel_city"].str.replace('Saint-Georges-de-Gréhaigne','Mont Saint Michel')
data["hotel_city"] = data["hotel_city"].str.replace('Ardevon','Mont Saint Michel')

#St Malo

data["hotel_city"] = data["hotel_city"].str.replace('Intra muros, Saint-Malo',"St Malo")
data["hotel_city"] = data["hotel_city"].str.replace('Sillon, Saint-Malo',"St Malo")

#Bayeux

data["hotel_city"] = data["hotel_city"].str.replace('Port-en-Bessin-Huppain','Bayeux')
data["hotel_city"] = data["hotel_city"].str.replace('Arromanches-les-Bains','Bayeux')

#Le Havre

data["hotel_city"] = data["hotel_city"].str.replace('Perret, Le Havre','Le Havre')

#Rouen

data["hotel_city"] = data["hotel_city"].str.replace('Centre-ville de Rouen, Rouen','Rouen')

#Paris

data["hotel_city"] = data["hotel_city"].str.replace('5e arr., Paris','Paris')
data["hotel_city"] = data["hotel_city"].str.replace('3e arr., Paris','Paris')
data["hotel_city"] = data["hotel_city"].str.replace('4e arr., Paris','Paris')
data["hotel_city"] = data["hotel_city"].str.replace('1er arr., Paris','Paris')
data["hotel_city"] = data["hotel_city"].str.replace('6e arr., Paris','Paris')

#Lille

data["hotel_city"] = data["hotel_city"].str.replace('Centre de Lille, Lille','Lille')
data["hotel_city"] = data["hotel_city"].str.replace('Vieux-Lille, Lille','Lille')

#Strasbourg 

data["hotel_city"] = data["hotel_city"].str.replace('Gare de Strasbourg, Strasbourg','Strasbourg')
data["hotel_city"] = data["hotel_city"].str.replace('Centre de Strasbourg - Petite France - Cathédrale, Strasbourg','Strasbourg')
data["hotel_city"] = data["hotel_city"].str.replace('Bourse-Esplanade, Strasbourg','Strasbourg')
data["hotel_city"] = data["hotel_city"].str.replace('Quartier Européen - Orangerie, Strasbourg','Strasbourg')

#Château du Haut Koenigsbourg

data["hotel_city"] = data["hotel_city"].str.replace('La Vancelle','Château du Haut Koenigsbourg')
data["hotel_city"] = data["hotel_city"].str.replace('Kintzheim','Château du Haut Koenigsbourg')
data["hotel_city"] = data["hotel_city"].str.replace('Rodern','Château du Haut Koenigsbourg')
data["hotel_city"] = data["hotel_city"].str.replace('Orschwiller','Château du Haut Koenigsbourg')
data["hotel_city"] = data["hotel_city"].str.replace('Bergheim','Château du Haut Koenigsbourg')
data["hotel_city"] = data["hotel_city"].str.replace('Saint-Hippolyte','Château du Haut Koenigsbourg')
data["hotel_city"] = data["hotel_city"].str.replace('Châtenois','Château du Haut Koenigsbourg')
data["hotel_city"] = data["hotel_city"].str.replace('Thannenkirch','Château du Haut Koenigsbourg')

#Colmar 

data["hotel_city"] = data["hotel_city"].str.replace('Centre-ville de Colmar, Colmar','Colmar')
data["hotel_city"] = data["hotel_city"].str.replace('Wintzenheim','Colmar')
data["hotel_city"] = data["hotel_city"].str.replace('Gueberschwihr','Colmar')
data["hotel_city"] = data["hotel_city"].str.replace('Wettolsheim','Colmar')
data["hotel_city"] = data["hotel_city"].str.replace('Husseren-les-Châteaux','Colmar')
data["hotel_city"] = data["hotel_city"].str.replace('Turckheim','Colmar')

#Besancon  

data["hotel_city"] = data["hotel_city"].str.replace('Besancon Old Town, Besançon','Besancon')
data["hotel_city"] = data["hotel_city"].str.replace('Besançon','Besancon')

#Dijon 

data["hotel_city"] = data["hotel_city"].str.replace('Centre-ville de Dijon, Dijon','Dijon')

#Annecy  

data["hotel_city"] = data["hotel_city"].str.replace("Centre-ville d'Annecy, Annecy","Annecy") 

#Grenoble 

data["hotel_city"] = data["hotel_city"].str.replace('Grenoble City Centre, Grenoble','Grenoble')

#Lyon

data["hotel_city"] = data["hotel_city"].str.replace('1er arr., Lyon','Lyon') 
data["hotel_city"] = data["hotel_city"].str.replace('2e arr., Lyon','Lyon')
data["hotel_city"] = data["hotel_city"].str.replace('5e arr., Lyon','Lyon')

#Marseille
data["hotel_city"] = data["hotel_city"].str.replace('Castellane et Préfecture, Marseille','Marseille')

#Gorges du Verdon

data["hotel_city"] = data["hotel_city"].str.replace('Castellane','Gorges du Verdon')  
data["hotel_city"] = data["hotel_city"].str.replace('Gréoux-les-Bains','Gorges du Verdon')
data["hotel_city"] = data["hotel_city"].str.replace('La Palud-sur-Verdon','Gorges du Verdon')
data["hotel_city"] = data["hotel_city"].str.replace('Riez','Gorges du Verdon')
data["hotel_city"] = data["hotel_city"].str.replace('Bauduen','Gorges du Verdon')
data["hotel_city"] = data["hotel_city"].str.replace('Moustiers-Sainte-Marie','Gorges du Verdon')
data["hotel_city"] = data["hotel_city"].str.replace('Baudinard','Gorges du Verdon')
data["hotel_city"] = data["hotel_city"].str.replace('Quinson','Gorges du Verdon')
data["hotel_city"] = data["hotel_city"].str.replace('La Martre','Gorges du Verdon')
data["hotel_city"] = data["hotel_city"].str.replace('Moissac-Bellevue','Gorges du Verdon')
data["hotel_city"] = data["hotel_city"].str.replace('Saint-Laurent-du-Verdon','Gorges du Verdon')
data["hotel_city"] = data["hotel_city"].str.replace('Comps-sur-Artuby','Gorges du Verdon')

#Bormes les Mimosas

data["hotel_city"] = data["hotel_city"].str.replace('Bormes-les-Mimosas','Bormes les Mimosas')
data["hotel_city"] = data["hotel_city"].str.replace("Saint-Clair Beach, Le Lavandou",'Bormes les Mimosas')
data["hotel_city"] = data["hotel_city"].str.replace('Le Lavandou','Bormes les Mimosas')

#Cassis

data["hotel_city"] = data["hotel_city"].str.replace('Carnoux-en-Provence','Cassis')


#Marseille

data["hotel_city"] = data["hotel_city"].str.replace('Saint-Charles, Marseille','Marseille')
data["hotel_city"] = data["hotel_city"].str.replace('Vieux Port - La Canebière, Marseille','Marseille')

#Aix en Provence

data["hotel_city"] = data["hotel_city"].str.replace("Centre historique d'Aix-en-Provence, Aix-en-Provence","Aix en Provence")
data["hotel_city"] = data["hotel_city"].str.replace("Aix-en-Provence","Aix en Provence")

#Avignon

data["hotel_city"] = data["hotel_city"].str.replace("Centre-ville d'Avignon, Avignon","Avignon")

#Uzès

data["hotel_city"] = data["hotel_city"].str.replace('Arpaillargues-et-Aureillac','Uzès')
data["hotel_city"] = data["hotel_city"].str.replace('Saint-Quentin-la-Poterie','Uzès')

#Nîmes 

data["hotel_city"] = data["hotel_city"].str.replace("Nimes City Centre, Nîmes","Nîmes")  

#Aigues Mortes

data["hotel_city"] = data["hotel_city"].str.replace('Aigues-Mortes Medieval City, Aigues-Mortes','Aigues Mortes')
data["hotel_city"] = data["hotel_city"].str.replace('Aigues-Mortes','Aigues Mortes')

#Saintes Maries de la mer

data["hotel_city"] = data["hotel_city"].str.replace("Les Saintes-Maries-de-la-Mer","Saintes Maries de la mer")

#Carcassonne   

data["hotel_city"] = data["hotel_city"].str.replace('Cité de Carcassonne, Carcassonne','Carcassonne')

#Ariège

data["hotel_city"] = data["hotel_city"].str.replace('Ax-les-Thermes','Ariège')
data["hotel_city"] = data["hotel_city"].str.replace('Tarascon-sur-Ariège','Ariège')
data["hotel_city"] = data["hotel_city"].str.replace('Aulus-les-Bains','Ariège')
data["hotel_city"] = data["hotel_city"].str.replace('Luzenac','Ariège')
data["hotel_city"] = data["hotel_city"].str.replace('Seix','Ariège')
data["hotel_city"] = data["hotel_city"].str.replace('Foix','Ariège')
data["hotel_city"] = data["hotel_city"].str.replace('Goulier-et-Olbier','Ariège')
data["hotel_city"] = data["hotel_city"].str.replace('Laroque-dʼOlmes','Ariège')
data["hotel_city"] = data["hotel_city"].str.replace('Montferrier','Ariège')
data["hotel_city"] = data["hotel_city"].str.replace('Roquefixade','Ariège')
data["hotel_city"] = data["hotel_city"].str.replace('Mirepoix','Ariège')
data["hotel_city"] = data["hotel_city"].str.replace('Mercus-Garrabet','Ariège')
data["hotel_city"] = data["hotel_city"].str.replace('Pamiers','Ariège')
data["hotel_city"] = data["hotel_city"].str.replace('Saint-Pierre-de-Rivière','Ariège')
data["hotel_city"] = data["hotel_city"].str.replace('Bélesta','Ariège')
data["hotel_city"] = data["hotel_city"].str.replace('Saint-Girons','Ariège')
data["hotel_city"] = data["hotel_city"].str.replace('Biert','Ariège')
data["hotel_city"] = data["hotel_city"].str.replace('Saint-Paul-de-Jarrat','Ariège')

#Toulouse

data["hotel_city"] = data["hotel_city"].str.replace('Centre de Toulouse, Toulouse','Toulouse')

#Biarritz

data["hotel_city"] = data["hotel_city"].str.replace('Centre de Biarritz, Biarritz','Biarritz')

#Bayonne

data["hotel_city"] = data["hotel_city"].str.replace('Lahonce','Bayonne')
data["hotel_city"] = data["hotel_city"].str.replace('Anglet','Bayonne')

#La Rochelle

data["hotel_city"] = data["hotel_city"].str.replace("Centre-ville de La Rochelle, La Rochelle","La Rochelle")

In [20]:
data['hotel_city'].unique()

array(['Mont Saint Michel', 'St Malo', 'Bayeux', 'Rouen', 'Le Havre',
       'Paris', 'Amiens', 'Lille', 'Strasbourg',
       'Château du Haut Koenigsbourg', 'Colmar', 'Dijon', 'Annecy',
       'Lyon', 'Grenoble', 'Besancon', 'Eguisheim', 'Gorges du Verdon',
       'Bormes les Mimosas', 'Cassis', 'Aix en Provence', 'Marseille',
       'Avignon', 'Nîmes', 'Aigues Mortes', 'Saintes Maries de la mer',
       'Collioure', 'Carcassonne', 'Ariège', 'Toulouse', 'Montauban',
       'Biarritz', 'Bayonne', 'Uzès', 'La Rochelle'], dtype=object)

In [21]:
data.to_csv("hotels_booking.csv")

In [22]:
kayak_data = pd.merge(df, data, left_on= "cities", right_on= "hotel_city")

In [23]:
kayak_data.shape

(773, 19)

In [24]:
kayak_data.head()

,ID,cities,city_lat,city_lon,felt_temp_j+1,felt_temp_j+2,felt_temp_j+3,felt_temp_j+4,felt_temp_j+5,felt_temp_j+6,felt_temp_j+7,felt_temp_avg,hotel_name,hotel_url,hotel_describe,hotel_ranking,hotel_city,hotel_lat,hotel_lon
0,1,Mont Saint Michel,48.635954,-1.51146,10.88,4.89,3.78,1.86,2.71,3.19,5.26,4.652857,Le Lithana,https://www.booking.com/hotel/fr/le-lithana.fr...,"Situé dans le centre-ville de Pontorson, l'hôt...",7.3,Mont Saint Michel,48.554097,-1.500629
1,1,Mont Saint Michel,48.635954,-1.51146,10.88,4.89,3.78,1.86,2.71,3.19,5.26,4.652857,Le Beauvoir,https://www.booking.com/hotel/fr/le-beauvoir-b...,Le Beauvoir est situé à 4 kilomètres du Mont-S...,7.7,Mont Saint Michel,48.597713,-1.513098
2,1,Mont Saint Michel,48.635954,-1.51146,10.88,4.89,3.78,1.86,2.71,3.19,5.26,4.652857,Hôtel Ariane & SPA,https://www.booking.com/hotel/fr/ariane-pontor...,Ce charmant hôtel est situé à la frontière ent...,7.6,Mont Saint Michel,48.552403,-1.507123
3,1,Mont Saint Michel,48.635954,-1.51146,10.88,4.89,3.78,1.86,2.71,3.19,5.26,4.652857,"Manoir de la Roche Torin, The Originals Relais...",https://www.booking.com/hotel/fr/le-manoir-de-...,"Situé à Courtils, à 10 km du Mont-Saint-Michel...",7.8,Mont Saint Michel,48.637273,-1.425062
4,1,Mont Saint Michel,48.635954,-1.51146,10.88,4.89,3.78,1.86,2.71,3.19,5.26,4.652857,"The Originals Boutique, Hôtel Les Quatre Salin...",https://www.booking.com/hotel/fr/lessalines.fr...,"L'établissement The Originals Boutique, Hôtel ...",7.9,Mont Saint Michel,48.596404,-1.589203


In [25]:
kayak_data.to_csv("kayak.csv")

In [27]:
hotel_cities = pd.read_csv('kayak.csv',lineterminator='\n')
hotel_cities.head()

,Unnamed: 0,ID,cities,city_lat,city_lon,felt_temp_j+1,felt_temp_j+2,felt_temp_j+3,felt_temp_j+4,felt_temp_j+5,felt_temp_j+6,felt_temp_j+7,felt_temp_avg,hotel_name,hotel_url,hotel_describe,hotel_ranking,hotel_city,hotel_lat,hotel_lon
0,0,1,Mont Saint Michel,48.635954,-1.51146,10.88,4.89,3.78,1.86,2.71,3.19,5.26,4.652857,Le Lithana,https://www.booking.com/hotel/fr/le-lithana.fr...,"Situé dans le centre-ville de Pontorson, l'hôt...",7.3,Mont Saint Michel,48.554097,-1.500629
1,1,1,Mont Saint Michel,48.635954,-1.51146,10.88,4.89,3.78,1.86,2.71,3.19,5.26,4.652857,Le Beauvoir,https://www.booking.com/hotel/fr/le-beauvoir-b...,Le Beauvoir est situé à 4 kilomètres du Mont-S...,7.7,Mont Saint Michel,48.597713,-1.513098
2,2,1,Mont Saint Michel,48.635954,-1.51146,10.88,4.89,3.78,1.86,2.71,3.19,5.26,4.652857,Hôtel Ariane & SPA,https://www.booking.com/hotel/fr/ariane-pontor...,Ce charmant hôtel est situé à la frontière ent...,7.6,Mont Saint Michel,48.552403,-1.507123
3,3,1,Mont Saint Michel,48.635954,-1.51146,10.88,4.89,3.78,1.86,2.71,3.19,5.26,4.652857,"Manoir de la Roche Torin, The Originals Relais...",https://www.booking.com/hotel/fr/le-manoir-de-...,"Situé à Courtils, à 10 km du Mont-Saint-Michel...",7.8,Mont Saint Michel,48.637273,-1.425062
4,4,1,Mont Saint Michel,48.635954,-1.51146,10.88,4.89,3.78,1.86,2.71,3.19,5.26,4.652857,"The Originals Boutique, Hôtel Les Quatre Salin...",https://www.booking.com/hotel/fr/lessalines.fr...,"L'établissement The Originals Boutique, Hôtel ...",7.9,Mont Saint Michel,48.596404,-1.589203


#### Visualization of all hotels in the 35 cities in France

In [29]:
fig3 = px.scatter_mapbox(hotel_cities, lat="hotel_lat", lon="hotel_lon", color="hotel_ranking",hover_name="hotel_name",
                        mapbox_style="carto-positron", zoom =4)
fig3.show()

#### Visualisations of hotels in top 5 cities for the next seven days

In [30]:
Top5cities[0:5]

,ID,cities,city_lat,city_lon,felt_temp_j+1,felt_temp_j+2,felt_temp_j+3,felt_temp_j+4,felt_temp_j+5,felt_temp_j+6,felt_temp_j+7,felt_temp_avg
18,19,Bormes les Mimosas,43.150697,6.341928,13.82,13.53,11.41,8.51,8.72,9.09,5.30,10.054286
33,34,Bayonne,43.493338,-1.475099,14.83,12.32,5.11,7.48,9.95,7.05,10.43,9.595714
32,33,Biarritz,43.483252,-1.559278,14.54,12.18,5.06,7.31,10.10,6.99,10.47,9.521429
27,28,Collioure,42.525050,3.083155,15.28,12.79,9.72,3.87,8.68,4.99,10.24,9.367143
19,20,Cassis,43.218178,5.553394,12.88,14.19,10.65,7.28,6.88,7.16,4.08,9.017143


In [31]:
# Hotels in Bormes les Mimosas

hotels_Bormes_les_Mimosas = hotel_cities[hotel_cities["cities"]== "Bormes les Mimosas"]

fig4 = px.scatter_mapbox(hotels_Bormes_les_Mimosas, lat="hotel_lat", lon="hotel_lon", color="hotel_ranking",hover_name="hotel_name", size="hotel_ranking",
                        mapbox_style="carto-positron", zoom = 12)
fig4.show()

In [37]:
# Hotels in Bayonne

hotels_Bayonne = hotel_cities[hotel_cities["cities"]== "Bayonne"]

fig5 = px.scatter_mapbox(hotels_Bayonne, lat="hotel_lat", lon="hotel_lon", color="hotel_ranking",hover_name="hotel_name", size="hotel_ranking",
                        mapbox_style="carto-positron", zoom = 12)
fig5.show()

In [38]:
# Hotels in Biarritz

hotels_Biarritz = hotel_cities[hotel_cities["cities"]== "Biarritz"]

fig6 = px.scatter_mapbox(hotels_Biarritz, lat="hotel_lat", lon="hotel_lon", color="hotel_ranking",hover_name="hotel_name", size="hotel_ranking",
                        mapbox_style="carto-positron", zoom = 14)
fig6.show()

In [43]:
# Hotels in Collioure

hotels_Collioure = hotel_cities[hotel_cities["cities"]== "Collioure"]

fig7 = px.scatter_mapbox(hotels_Collioure, lat="hotel_lat", lon="hotel_lon", color="hotel_ranking",hover_name="hotel_name",size="hotel_ranking",
                        mapbox_style="carto-positron", zoom = 15)
fig7.show()

In [44]:
# Hotels in Cassis

hotels_Cassis = hotel_cities[hotel_cities["cities"]== "Cassis"]

fig8 = px.scatter_mapbox(hotels_Cassis, lat="hotel_lat", lon="hotel_lon", color="hotel_ranking",hover_name="hotel_name", size="hotel_ranking",
                        mapbox_style="carto-positron", zoom = 13)
fig8.show()

#### Part 4 and Part 5 in the second notebook called : Projet_kayak_part2